In [1]:
!pip install sktime
!pip install tsfresh
import sktime
import numpy as np
import pandas as pd
from sktime.utils.load_data import load_from_arff_to_dataframe
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

from sktime.classification.compose import TimeSeriesForestClassifier
from sktime.utils.time_series import time_series_slope

     |████████████████████████████████| 5.6MB 5.7MB/s 
     |████████████████████████████████| 6.8MB 44.2MB/s 
     |████████████████████████████████| 9.5MB 38.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
     |████████████████████████████████| 92kB 4.2MB/s 
     |████████████████████████████████| 665kB 8.9MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 102kB 6.7MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=6c90e71755bbb8dd800d73a7143ef89a83e80a6fc0eefdb428466de01e400073
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
  Created wheel for locket: filename=locket-0.2.0-cp36-

In [2]:
namesCol = ['subject', 'group', 'condition', 'trial', 'channel']

for q in range(256):
  strVar = 'v' + str(q)
  namesCol.append(strVar)

df = pd.read_csv('eeg_small_uci.csv', header = 0, names = namesCol)
col_name = list(df.columns)
trans_df = pd.DataFrame(data = df, columns = col_name)

In [3]:
trans_df.head()

,subject,group,condition,trial,channel,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,...,v216,v217,v218,v219,v220,v221,v222,v223,v224,v225,v226,v227,v228,v229,v230,v231,v232,v233,v234,v235,v236,v237,v238,v239,v240,v241,v242,v243,v244,v245,v246,v247,v248,v249,v250,v251,v252,v253,v254,v255
0,a_1_co2a0000364,a,S1,0,FP1,-8.921,-8.433,-2.574,5.239,11.587,14.028,11.587,6.704,1.821,-1.109,-2.085,-1.597,0.356,2.309,2.797,2.309,0.356,-0.132,-0.132,0.356,0.844,1.333,2.797,6.215,9.633,10.610,6.704,-0.621,-5.992,-6.480,-1.597,4.262,7.680,5.727,1.821,...,14.516,14.028,9.633,5.727,4.262,5.727,7.192,7.192,5.727,4.751,6.704,10.610,13.051,13.051,9.633,6.704,6.704,8.657,10.610,9.145,4.751,-0.132,-2.574,-1.109,2.309,6.215,8.657,10.610,12.075,12.563,10.610,8.169,4.751,2.797,2.797,3.286,3.774,4.262,5.727,8.169
1,a_1_co2a0000364,a,S1,0,FP2,0.834,3.276,5.717,7.670,9.623,9.623,8.647,5.229,1.322,-2.096,-4.049,-5.025,-5.025,-5.025,-4.537,-4.049,-1.607,2.299,6.205,8.647,9.135,7.670,6.205,5.717,6.205,4.252,-0.631,-7.955,-11.861,-10.396,-2.584,6.205,10.111,6.694,0.346,...,12.553,9.135,3.764,-0.142,-0.631,2.787,6.694,8.647,8.158,6.694,6.205,6.694,6.694,5.229,2.787,3.276,6.694,10.600,11.576,7.182,-0.142,-5.514,-5.025,1.322,8.158,11.088,9.623,5.717,3.764,7.182,12.065,15.483,12.553,5.229,-2.584,-6.490,-4.049,2.787,9.623,13.041
2,a_1_co2a0000364,a,S1,0,F7,-19.847,-12.522,1.149,14.821,20.681,17.263,8.962,0.173,-3.733,-2.268,0.661,2.614,3.103,2.614,3.103,4.567,7.009,7.985,7.009,4.079,0.173,-2.268,-0.804,3.591,8.962,11.403,9.939,4.567,-1.780,-5.198,-3.245,2.614,8.962,11.403,8.962,...,18.728,21.169,17.263,10.915,7.009,8.962,13.357,15.798,12.380,5.544,0.173,1.149,7.009,13.357,16.286,15.310,13.845,15.310,18.239,18.728,14.821,7.497,2.126,2.126,6.521,11.403,13.357,13.357,14.333,16.774,19.216,17.751,10.915,2.126,-4.222,-4.710,0.661,7.009,11.403,12.868
3,a_1_co2a0000364,a,S1,0,F8,8.148,1.801,-2.594,-4.547,-5.035,-5.524,-4.547,-0.641,5.707,10.590,9.125,1.801,-6.500,-9.918,-6.500,-0.641,4.242,4.730,1.312,-2.594,-6.500,-7.477,-4.059,4.242,14.008,19.379,15.961,3.754,-9.918,-16.266,-12.360,-3.082,2.777,-0.641,-8.942,...,4.730,9.613,11.078,9.125,5.707,2.777,0.336,-3.082,-6.012,-6.500,-2.106,5.219,10.101,7.660,-1.129,-9.430,-10.406,-2.106,10.101,17.914,15.961,7.172,-2.106,-6.500,-5.524,-4.059,-3.571,-3.082,0.336,8.148,16.449,20.355,16.937,7.660,-2.594,-9.430,-10.406,-7.477,-1.617,4.242
4,a_1_co2a0000364,a,S1,0,AF1,-2.146,-2.146,-1.658,-0.682,2.248,5.178,6.643,5.178,1.760,-1.658,-3.611,-1.658,1.272,3.225,2.736,0.295,-1.658,-0.682,1.760,3.713,3.225,2.248,2.736,4.690,6.643,6.643,3.225,-1.658,-4.100,-3.123,0.295,3.713,4.690,2.736,0.783,...,10.549,10.549,7.131,2.736,0.295,0.295,2.248,5.666,7.131,7.131,5.178,3.225,2.248,3.225,4.201,5.666,6.154,4.201,2.248,0.295,-0.682,0.295,1.760,3.225,3.713,4.201,3.225,2.248,0.295,-1.170,-1.658,0.295,2.736,4.201,3.713,1.272,-0.682,-0.193,3.225,7.619


In [4]:
# Get Y-target -df
y = trans_df['condition']

# Drop target variable and get X-feature- df
X = trans_df.drop(['condition'], axis = 1)

In [5]:
# Splitting the dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(2880, 260) (2880,) (960, 260) (960,)


In [6]:
# Label Encoding

from sklearn import preprocessing

cat_cols = ['subject', 'group', 'trial']
enc = preprocessing.LabelEncoder()

for col in cat_cols:
    X_train[col] = X_train[col].astype('str')
    X_test[col] = X_test[col].astype('str')
    X_train[col] = enc.fit_transform(X_train[col])
    X_test[col] = enc.transform(X_test[col])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [7]:
#isolate the time series
X_train_timedata = X_train[X_train.columns[4:260]]
X_test_timedata = X_test[X_test.columns[4:260]]



X_train_timedata['combine'] = X_train_timedata.values.tolist()
X_test_timedata['combine'] = X_test_timedata.values.tolist()
X_train_timedata = X_train_timedata['combine']
X_test_timedata = X_test_timedata['combine']

#convert to dataframe
X_train_timedata = X_train_timedata.to_frame()
X_test_timedata = X_test_timedata.to_frame()




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
ts_train = pd.Series(X_train_timedata['combine'].values, index=X_train_timedata.index)
X_ts_train = ts_train.to_frame()

ts_test = pd.Series(X_test_timedata['combine'].values, index=X_test_timedata.index)
X_ts_test = ts_test.to_frame()

In [9]:
for row_num in range(0,X_ts_train.shape[0]):
  series = pd.Series(X_ts_train.iat[row_num,0])
  X_ts_train.iat[row_num,0] = series

for row_num in range(0,X_ts_test.shape[0]):
  series2 = pd.Series(X_ts_test.iat[row_num,0])
  X_ts_test.iat[row_num,0] = series2

In [10]:
X_ts_test.head()

,0
746,0 -5.107 1 -3.642 2 -1.200 3 ...
3837,0 0.244 1 -0.244 2 -1.221 3 -...
3218,0 -4.384 1 -5.361 2 -7.314 3 ...
2778,0 2.716 1 2.716 2 1.740 3 -...
1904,0 -2.502 1 -3.967 2 -4.456 3 -...


In [11]:
X_ts_train.head()

,0
1034,0 -1.943 1 -3.896 2 -6.826 3 ...
198,0 2.563 1 2.075 2 1.099 3 ...
2736,0 2.279 1 3.743 2 2.279 3 ...
654,0 10.254 1 7.324 2 3.418 3 ...
471,0 3.520 1 4.496 2 4.985 3 ...


# K-Nearest Neighbors



In [17]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
knn = KNeighborsTimeSeriesClassifier(n_neighbors=12, metric="dtw")
knn.fit(X_ts_train, y_train)

print("accuracy score:")
knn.score(X_ts_test, y_test)




accuracy score:


0.8833333333333333

In [18]:
y_pred = knn.predict(X_ts_test)
accuracy_score(y_test, y_pred)

0.8833333333333333